In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# hyper parameters
learning_rate = 0.001
training_epochs = 10
batch_size = 100
num_models = 2

In [3]:
class Model:
    
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self.buildNet()
    
    def buildNet(self):
        with tf.variable_scope(self.name):
            self.training = tf.placeholder(tf.bool)
            self.X = tf.placeholder(tf.float32, [None, 784])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            
            # Layer1
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[4,4], padding='VALID', activation=tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], padding='VALID', strides=1)
            dropout1 = tf.layers.dropout(inputs=pool1, rate=0.3, training=self.training)
            
            # Layer2
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[4,4], padding='VALID', activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[4,4], padding='VALID', strides=1)
            dropout2 = tf.layers.dropout(inputs=pool2, rate=0.3, training=self.training)
            
            # Dense Layer
            flat = tf.reshape(dropout2, [-1, 18 * 18* 64])
            dense = tf.layers.dense(inputs=flat, units= 6 * 6 * 8, activation=tf.nn.relu)
            dropout3 = tf.layers.dropout(inputs=dense, rate=0.5, training=self.training)
            
            self.logit = tf.layers.dense(inputs=dropout3, units=10)
            
        with tf.name_scope("cost"):
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logit, labels=self.Y))
            self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
            cost_hist = tf.summary.scalar(self.name+"_cost", self.cost)
        
        with tf.name_scope("prediction"):
            correct_prediction = tf.equal(tf.argmax(self.logit, 1), tf.argmax(self.Y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            accuracy_hist = tf.summary.scalar(self.name+"_accuracy", self.accuracy)
        
    def train(self, x_data, y_data, training=True):
        return sess.run([self.cost, self.optimizer], feed_dict={self.X:x_data, self.Y:y_data, self.training:training})
    
    def predict(self, x_test, training=False):
        return self.sess.run(self.logit, feed_dict={self.X: x_test, self.training: training})
    
    def get_accuracy(self, x_data, y_data, training=False):
        return sess.run(self.accuracy, feed_dict={self.X:x_data, self.Y:y_data, self.training:training})

In [4]:
sess = tf.Session()

models =[]

for m in range(num_models):
    models.append(Model(sess, "model"+str(m+1)))

sess.run(tf.global_variables_initializer())
'''
summary = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs/mnist_problem/mnist_cnn_ensemble')
writer.add_graph()
'''
#train
for epoch in range(training_epochs):
    avg_cost_list = [0 for i in range(len(models))]
    
    for step in range(mnist.train.num_examples / batch_size):
        x_data, y_data = mnist.train.next_batch(batch_size=batch_size)
        
        for m_idx, m in enumerate(models):
            c, _ = m.train(x_data, y_data)
            avg_cost_list[m_idx] += c / (mnist.train.num_examples / batch_size)
                                        
    predictions = np.zeros([mnist.test.num_examples, 10])
                                         
    for m_idx, m in enumerate(models):
        print m_idx, 'Accuracy:', m.get_accuracy(mnist.test.images, mnist.test.labels)
        p = m.predict(mnist.test.images)
        predictions += p

    ensemble_correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
    ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))
    print 'Ensemble accuracy:', sess.run(ensemble_accuracy)
            

0 Accuracy: 0.9873
1 Accuracy: 0.9866
Ensemble accuracy: 0.9883


KeyboardInterrupt: 